In [1]:
import json
import jsonlines
import sys

# dirs = ["..", "/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/my_inferencing/create_prompt_llm"]
# for _dir in dirs:
#     if _dir not in sys.path:
#         sys.path.append(_dir)
import prompt_rag

search_engine = "brave"

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)


In [2]:
# prompt_rag.update_train_search_llm( 
#     "SOLAR-10.7B-Instruct-v1.0", "solar", port=8001, 
#     data_search=data_search, data_search_llm=data_search_llm) 



合并多个文件的先验知识

In [3]:
# # 对prior_knowledge做修改
# sort = False
# prior_knowledge_list = []

# K = 5
# prior_knowledge_version = "1"
# prior_knowledge_version_adjust = prior_knowledge_version + ".2"
# model_name = "mixtral"


# for i, item in enumerate(data_search_llm):
#     prior_knowledge = item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"]
    
#     find_list = [
#         "Information Expansion and Summary:",
#         # "Information Summary:",
#         # "Expanded Information Summary:",
#         "Claim Analysis:",
#         # "Claim Verification",
#         # "Claim Analysis and Judgement:"
#     ]

#     for s in find_list:
#         pos = prior_knowledge.find(s)
#         if pos == -1:
#             continue
        
#         num_extra = 2
#         if pos!=0 and prior_knowledge[pos-1] == '\n':
#             pos -= 1
#             num_extra += 1
#         prior_knowledge = prior_knowledge[:pos] + prior_knowledge[pos + len(s) + num_extra :]

  
#     item[f"prior_knowledge_{model_name}_v{prior_knowledge_version_adjust}_K={K}"] = prior_knowledge


In [4]:
# bing search llm
# v2: k = 5
# v3: 不对时间排序

# brave search llm

# Solar
# v1: k = 5
# v2: k = 5 随机选取

# Mixtral
# v1: k = 5
# v1.1: 删除结构化内容（Information Expansion and Summary、Claim Analysis）

sort = False
prior_knowledge_list = []

K = 5
prior_knowledge_version = "1"
model_name = "mixtral"


# with open(f"train_search_llm_tmp.json", "r") as f:
#     prior_knowledge_list = json.load(f)

# for i, item in enumerate(data_search_llm):
#     if item["claim"] != prior_knowledge_list[i]["claim"].strip():
#         print(i)
#         print(item["claim"])
#         print(prior_knowledge_list[i]["claim"])
#         raise Exception()
#     else:
#         item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"] = prior_knowledge_list[i][f"prior_knowledge_{model_name}"]

# data_search_llm[0]


In [5]:
# prompt_rag.save_search_llm(data_search_llm, search_engine)


创建数据（带有先验知识的Prompt）以微调LLM

In [6]:
# dict_list = []

# for i, item in enumerate(data_search_llm):
    
#     if int(item["label"]) != 1:
#         prompt = prompt_rag.get_prompt_with_prior_knowledge(
#             item["claim"], 
#             search_engine,
#             data_search[i][f"{search_engine}_search_results"], 
#             item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"], 
#             K=K,
#             claim_date=item["date"],
#             sort=sort,
#             ids=None 
#         )
#         label = "TRUE." if int(item["label"]) == 2 else "FALSE."
#         dict_list.append({"query": prompt, "response": label})
# print(dict_list[0]["query"])
# len(dict_list), dict_list[:2]


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-08-13
Content: Illinois Strengthens Face Mask Rules in Businesses

PRIOR KNOWLEDGE:
Information 1:
Publication date: 2013-08-06
Title: Illinois Strengthens Face Mask Rules in Businesses - WebMD
Content:
COVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.
Information 2:
Publication date: None
Title: press-release
Content:
Illinoisans can resume activities without wearing a mask indoors on February 28th except where required by federal, state, local, tribal, or territorial laws, rules, and regulations, including local business and workplace guidance. Federa

(12192,
 [{'query': 'According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-08-13\nContent: Illinois Strengthens Face Mask Rules in Businesses\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: 2013-08-06\nTitle: Illinois Strengthens Face Mask Rules in Businesses - WebMD\nContent:\nCOVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.\nInformation 2:\nPublication date: None\nTitle: press-release\nContent:\nIllinoisans can resume activities without wearing a mask indoors on February 28th except where required by federal, state, local, tribal, or territorial laws, rules, and regulations, including local bu

In [7]:
# LLM_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/"
# data_version = "1"
# with jsonlines.open(
#     LLM_dir + \
#     f"swift/examples/pytorch/llm/my_data/with_{model_name}_info/{search_engine}/data{data_version}.jsonl", mode="w") as file_jsonl:
#     for line in dict_list:
#         file_jsonl.write(line)


测试模型的先验知识生成效果：一次提问

In [1]:
import torch
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
# model_type = CustomModelType.solar_instruct_10_7b

llm_engine = get_vllm_engine(
    model_type, 
    torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=1,
    
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=4096,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True,
)



2024-03-28 11:10:32,264 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-03-28 11:10:32,267 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-03-28 11:10:32,305 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidde

WARNING 03-28 11:10:32 config.py:618] Casting torch.bfloat16 to torch.float16.
WARNING 03-28 11:10:32 config.py:193] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 03-28 11:10:32 llm_engine.py:87] Initializing an LLM engine with config: model='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', tokenizer='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=42)


OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacty of 23.65 GiB of which 10.06 MiB is free. Including non-PyTorch memory, this process has 23.63 GiB memory in use. Of the allocated memory 22.70 GiB is allocated by PyTorch, and 358.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
K = 5
def get_id(claim):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            return i

i = 0
# i = get_id("The Government of Mexico City offers $1,000 weekly if you stay at home.")

claim = data_search[i]["claim"]
search_results = data_search[i][f"{search_engine}_search_results"]

model_name = 'mixtral'
prompt_list = [
    prompt_rag.get_prompt_for_generating_prior_knowledge(
        claim, data_search[i]["date"], search_engine, search_results, model_name,
        K=K, sort=False, 
        # ids=data_search[i]["random_ids"],
        ids=None
    ),
]

request_list = [{'query': prompt} for prompt in prompt_list]

print(prompt_list[0])

resp_list = get_resp_list(request_list)
print(resp_list[0]["response"].strip())
print()


Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.

CLAIM:
Publication date: 2020-08-13
Content: Illinois Strengthens Face Mask Rules in Businesses

INFORMATION:
Information 1:
Publication date: 2013-08-06
Title: Illinois Strengthens Face Mask Rules in Businesses - WebMD
Content:
COVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, preve

100%|██████████| 1/1 [00:10<00:00, 10.17s/it]

Information Expansion and Summary:

Information 1, published on August 6, 2013, is not related to the claim, as it discusses COVID-19 symptoms, risk factors, and prevention measures but does not mention any changes to face mask rules in Illinois businesses.

Information 2, the press release, was published on an unspecified date. It announces that Illinois will lift the indoor mask requirement for the State of Illinois on February 28, 2022, except where required by federal, state, local, tribal, or territorial laws, rules, and regulations. Businesses and organizations can still have their own mask requirements. The decision is based on decreasing COVID-19 cases and increasing vaccination rates.

Information 3, the FAQs on Mask and Vaccine Requirements, was published on September 3, 2021. It states that Executive Order 21-22 requires all individuals over the age of 2 to wear a face covering in indoor public places and imposes vaccine requirements for certain groups.

Information 4, the F